In [ ]:

import numpy as np 
import pandas as pd
import json

In [ ]:

meta = pd.read_csv('/content/drive/My Drive/data/the-movies-dataset/movies_metadata.csv') # 내파일을 읽어준다. 그때 0-1열을 가져오고 그 이름을 'Cust_Id', 'Rating'으로 짓는다.


In [ ]:
meta.head()


In [ ]:
meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta = meta.rename(columns={'id':'movieId'})
meta = meta[meta['original_language'] == 'en']
meta.head()

In [ ]:
ratings = pd.read_csv('/content/drive/My Drive/data/the-movies-dataset/ratings_small.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()
len(ratings)

In [ ]:
ratings.describe()


In [ ]:
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')

In [ ]:
type(meta['genres'])

In [ ]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])

    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres) #각 행에대해 함수를 적용하려면 apply를 써야 된다.

meta.head()

In [ ]:
# 데이터를 합치는데 inner조인으로 합친다.
data = pd.merge(ratings, meta, on='movieId', how='inner')

data.head()

In [ ]:
# 피벗테이블로 만드는데 index는 유저아이디고 컬럼값은 영화의 제목이다.
# 여기서 영화를 보고 평점을 줬는지에따라 Nan이나 점수를 준다. 
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')

matrix.head(20)

In [ ]:
GENRE_WEIGHT = 0.1

def pearsonR(s1, s2):  # 피어슨 상관관계를 만드는 함수
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))
    # 분자가 클수록 상관관계가 높다.
        # 유저가 넣은 영화 / 피벗테이블 /반환값 /장르별 가중치를 할지 선택
def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]
    # 값을 넣은 movie와 meta가 같을 때  origin_title의 장르를 가져온다.
    result = []
    for title in matrix.columns:
        if title == input_movie: # 타이틀과 input_movie가 같으면 넘긴다.
            continue

        # rating comparison
        # 매트릭스의 컬럼이름으로 루프를 돈다. title은 그 제목이고
        # 여기서 매트릭스의 타이틀이름과 inputmovice 를 넣고 피어슨 상관관계를 본다.
        cor = pearsonR(matrix[input_movie], matrix[title]) #피어슨 선형관계
        #여기서 cor 값을 구하고 

        # genre comparison (장르별 가중치를 두는 작업)
        if similar_genre and len(input_genres) > 0: # 장르별 가중치가 있고 장르가 있을 경우
            #루프 돌고있느 matirx의 컬럼명 값이 영화 제목을 만나면 장르값을 저장한다.
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]
            # np.sum으로 리스트에 있는 값들을 합친다?
            # 찾는 값이 있는지에 따라 true false를 반환한다. 왼쪽에 오른쪽 값이 있는지 확인한다.
            # 그에따라 true 개수를 세서 같은게 몇개있는지 분석한다.
            same_count = np.sum(np.isin(input_genres, temp_genres))

            #선형관계에서 나온 계수에 장르별 가중치를 더해준다.
            cor += (GENRE_WEIGHT * same_count)
        
        if np.isnan(cor): # 값이 없으면 넘기고
            continue
        else: #값이 있으면 result라는 리스트에 값을 넣는다.
            result.append((title, '{:.2f}'.format(cor), temp_genres))
    # 정렬해준다.
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

In [ ]:

recommend_result = recommend('The Dark Knight', matrix, 10, similar_genre=True)

pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])